The purpose of this file is to randomly assign each customer from the customer_churn_business_dataset.csv to an account_id from the ravenstack_accounts.csv.

We will follow the constraint of seats for each account.
For example, 

account_id = 1
seats = 3

All 3 seats will be randomly filled by a customer_id from the customer table.

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import os
import badgers # For generating realistic data quality issues from our Kaggle Dataset
from badgers.generators.tabular_data.missingness import MissingCompletelyAtRandom
from badgers.generators.tabular_data.outliers.distribution_sampling import *

# Create generator object with seed = 3219087321 for reproducibility
rng = np.random.default_rng(3219087321)

os.chdir('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Mess')
import MessGenerator 

In [2]:
## Read the file

os.chdir('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Glacier-Product-Release-Impact-Analysis\\raw_data')

try:
    accts = pd.read_csv("ravenstack_accounts.csv")
except FileNotFoundError:
    raise ValueError(f"Accounts csv not found")

try:
    customers = pd.read_csv("customer_churn_business_dataset.csv")
except FileNotFoundError:
    raise ValueError(f"Users csv not found")


### Validation

<!-- Ensure that our inputs and assumptions are correct so that the code can behave correctly -->



In [3]:
## Validate the accts table

# Set A - Set B -> What is in Set A, not in B. 
req_cols = {'account_id', 'seats'}
missing = req_cols - set(accts.columns)


# If missing is not empty, then we are missing those columns. 
if missing:
    raise ValueError(f"Accounts table missing: {missing}")


## Validate the users table
if "customer_id" not in customers.columns:
    raise ValueError("Users table missing customer_id")


In [4]:
## Ensure the seats column is an integer
accts['seats'] = pd.to_numeric(accts['seats'], errors="raise")


# If any seats are negative, raise error and return df with seats and acct_id
if (accts['seats'] < 0).any(): 
    neg_seats = accts.loc[accts['seats'] < 0, ['account_id', 'seats']]
    raise ValueError(f"{neg_seats} has negative seats")

# Check for non-integer seats
if (accts['seats'] % 1 != 0).any():
    float_seats = accts.loc[accts['seats'] % 1 != 0, ['account_id', 'seats']]
    raise ValueError(f"{float_seats} has non-integer seats")

# Check to see if # seats > # users
num_seats = int(accts['seats'].sum())
num_cust = len(customers['customer_id'])

seat_diff = num_seats - num_cust

# Seats > num_cust is allowed since we can have unassigned seats
# Seats < num_cust is not allowed since each customer must have a company assigned
if num_seats > num_cust:
    print(f"There are {seat_diff} more seats than users")
elif num_seats < num_cust:
    print(f"There are {seat_diff * -1} less seats than users")
    raise ValueError("There are not enough seats")
else:
    print(f"There are equal number of seats and number of users")

There are 280 more seats than users


## Randomly assign acct_id to each customer_id

In [5]:
# Create array with acct_id * # seats
# Ex: acct_id = A, seat = 3 -> [A,A,A]; 3 'A' seats to assign

rng_accts = np.repeat(accts['account_id'].to_numpy(), 
                      accts['seats'].to_numpy()
)

# Shuffle all the acct_ids in-place
rng.shuffle(rng_accts)

# If there are more seats than users, restrict the # of ids that will be assigned to be equal to num_cust
assign_ids = rng_accts[:num_cust]
customers["account_id"] = assign_ids

customers["account_id"]


0       A-bd4708
1       A-068fc6
2       A-4f18f0
3       A-ac85cd
4       A-66224b
          ...   
9995    A-1b707d
9996    A-1619f8
9997    A-b48f73
9998    A-2e3bad
9999    A-7f8241
Name: account_id, Length: 10000, dtype: object

### Introduce Data Quality Defects 


In [ ]:
# Generate missing values
trf = MissingCompletelyAtRandom(random_generator=rng)
accounts, _ = trf.generate(accts.copy(), y=None, percentage_missing=0.067)
accounts['is_trial'] = accts['is_trial']

# Generate outliers
trf = ZScoreSamplingGenerator(random_generator=rng)

# Filter to only numeric columns
num_cols = accounts.select_dtypes(include=[np.number]).columns

# Generate Outliers
outliers, _ = trf.generate(accounts[num_cols].copy(), y=None, n_outliers = 34)
accounts_out = accounts.copy()
idx_out = rng.choice(accounts_out.index.to_numpy(), size = len(outliers), replace=False)

accounts_out.loc[idx_out, num_cols] = outliers

# Generate duplicates
MessGenerator = MessGenerator.MessUpGenerator(random_generator = rng)
accounts, _ = MessGenerator.generate_dupes(accounts_out)



c:\Users\henry\AppData\Local\Programs\Python\Python312\Lib\site-packages\badgers\generators\tabular_data\missingness.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X.iloc[rows, col] = np.nan
c:\Users\henry\AppData\Local\Programs\Python\Python312\Lib\site-packages\badgers\generators\tabular_data\missingness.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X.iloc[rows, col] = np.nan


In [7]:
accounts
accounts.shape

(600, 10)

In [ ]:
# Generate Inconsistent Formatting
accounts, _ = MessGenerator.generate_incon_format(accounts, exclude_cols=("account_id", "account_name"))

In [9]:
accounts

,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,A-2e4581,Company_0,EdTech,US,10/16/2024,partner,Basic,9.000000,False,False
1,A-43a9e3,Company_1,FinTech,IN,8/17/2023,other,Basic,18.000000,False,True
2,A-0a282f,Company_2,DevTools,US,8/27/2024,organic,Basic,NaN,False,False
3,A-1f0ac7,Company_3,HealthTech,NaN,8/27/2023,other,Basic,24.000000,True,False
4,NaN,Company_4,None,US,10/27/2024,event,Enterprise,96.337312,False,True
...,...,...,...,...,...,...,...,...,...,...
595,NaN,Company_100,Cybersecurity,US,11/1/2024,event,Enterprise,1.000000,False,False
596,A-4814a3,Company_337,Cybersecurity,IN,9/17/2024,event,Basic,87.000000,False,True
597,A-92a333,Company_116,FinTech,US,3/24/2023,event,Enterprise,98.042606,False,False
598,A-1ac5e0,Company_126,DevTools,US,10/20/2024,event,Basic,33.000000,False,NaN
